In [10]:
import pandas as pd
import hydrofunctions as hf
from datetime import datetime
import matplotlib.pyplot as plt

# --- 1. Define station and dates ---
station_id = "12350250"  # Bitterroot River near Darby
start_date = "2000-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")  # Use current date

# --- 2. Retrieve daily discharge data from NWIS ---
# The parameter code '00060' is for discharge in cubic feet per second.
try:
    nwis_data = hf.NWIS(station_id, 'dv', start_date, end_date, parameterCd='00060')
    print(f"Data retrieval successful for station {station_id}.")
except Exception as e:
    print(f"Error retrieving data for station {station_id}: {e}")
    exit()

# --- 3. Convert to DataFrame ---
df_discharge = nwis_data.df()

# --- 4. Inspect the DataFrame ---
print("\n--- DataFrame Columns ---")
print(df_discharge.columns)

print("\n--- DataFrame Head ---")
print(df_discharge.head())

print("\n--- DataFrame Info ---")
print(df_discharge.info())

# --- 5. Identify the Correct Column to Rename ---
# Find the discharge column (usually contains '00060')
discharge_columns = [col for col in df_discharge.columns if '00060' in col]
print(f"\nDischarge Columns Found: {discharge_columns}")

if not discharge_columns:
    print("No discharge columns found. Please verify the parameter code and station ID.")
    exit()

# Assuming the first matching column is the desired one
discharge_col = discharge_columns[0]

# --- 6. Rename the Discharge Column ---
df_discharge.rename(columns={discharge_col: "Discharge_cfs"}, inplace=True)

# --- 7. Clean the DataFrame ---
df_discharge = df_discharge[['Discharge_cfs']].copy()
df_discharge.dropna(inplace=True)

# --- 8. Confirm the Structure After Renaming ---
print("\n--- Renamed DataFrame Head ---")
print(df_discharge.head())

print("\n--- Renamed DataFrame Description ---")
print(df_discharge.describe())

Requested data from https://waterservices.usgs.gov/nwis/dv/?format=json%2C1.1&sites=12350250&parameterCd=00060&startDT=2000-01-01&endDT=2025-01-13
Data retrieval successful for station 12350250.

--- DataFrame Columns ---
Index(['USGS:12350250:00060:00003', 'USGS:12350250:00060:00003_qualifiers'], dtype='object')

--- DataFrame Head ---
                           USGS:12350250:00060:00003  \
datetimeUTC                                            
2000-07-01 00:00:00+00:00                     1200.0   
2000-07-02 00:00:00+00:00                     1210.0   
2000-07-03 00:00:00+00:00                     1140.0   
2000-07-04 00:00:00+00:00                     1000.0   
2000-07-05 00:00:00+00:00                      855.0   

                          USGS:12350250:00060:00003_qualifiers  
datetimeUTC                                                     
2000-07-01 00:00:00+00:00                                    A  
2000-07-02 00:00:00+00:00                                    A  
2000-07-

In [13]:
# 1) Resample with 'A' instead of 'Y'
annual_mean_flow = df_discharge.resample('A').mean()
annual_mean_flow.index = annual_mean_flow.index.year  # convert DateTimeIndex to just the year

print("\n--- Annual Mean Flow (cfs) ---")
print(annual_mean_flow)

# 2) Plot as a bar chart
plt.figure(figsize=(10, 5))
plt.bar(annual_mean_flow.index, annual_mean_flow['Discharge_cfs'], color='skyblue')
plt.xlabel('Year')
plt.ylabel('Discharge (cfs)')
plt.title('Annual Mean Flow (cfs) - Bitterroot River')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 3) Save the figure (handles non-interactive environments)
plt.savefig('results/figures/annual_mean_flow.png')



--- Annual Mean Flow (cfs) ---
             Discharge_cfs
datetimeUTC               
2000            372.130435
2001            386.195652
2002           2114.377049
2003           2570.306011
2004            696.771739
2005            386.782609
2006            426.402174
2007            351.565217
2008           1311.054348
2009            928.119565
2010           1612.565421
2011           3152.799065
2012           1930.862745
2013           1183.322368
2014           2988.416327
2015           1565.636735
2016           1504.229091
2017           2648.742857
2018           3110.833333
2019           2009.428571
2020           2550.506122
2021           1976.167347
2022           2137.567347
2023           1539.730612
2024           1333.081967


/var/folders/vp/1c12nhtx09ng562vszvlczthvwd07y/T/ipykernel_26894/3896786481.py:2: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_mean_flow = df_discharge.resample('A').mean()
